In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import math

In [2]:
port = pd.read_csv("Portfolio.csv")
prices = pd.read_csv("DailyPrices.csv")

In [3]:
portA = port.loc[port['Portfolio'] == "A"][["Stock", "Holding"]]
portA.Holding = portA.Holding / sum(portA.Holding)
portB = port.loc[port['Portfolio'] == "B"][["Stock", "Holding"]]
portB.Holding = portB.Holding / sum(portB.Holding)
portC = port.loc[port['Portfolio'] == "C"][["Stock", "Holding"]]
portC.Holding = portC.Holding / sum(portC.Holding)
port = port[["Stock", "Holding"]]
port.Holding = port.Holding / sum(port.Holding)

In [4]:
portA = portA.set_index('Stock')
portB = portB.set_index('Stock')
portC = portC.set_index('Stock')
port = port.set_index('Stock')

In [5]:
pricesA = prices[list(portA.index)]
pricesB = prices[list(portB.index)]
pricesC = prices[list(portC.index)]
prices = pd.concat([pricesA, pricesB, pricesC])

In [6]:
returnsA = pricesA.pct_change()[1:]
returnsB = pricesB.pct_change()[1:]
returnsC = pricesC.pct_change()[1:]
returns = prices.pct_change()[1:]

In [7]:
returns

,AAPL,TSLA,FB,JPM,HD,BAC,XOM,AVGO,PEP,TMO,...,IBM,PLD,TGT,DE,MMM,F,LRCX,MO,LMT,TFC
1,0.001474,0.032571,0.003228,-0.007844,0.017838,-0.005521,-0.018168,0.009344,-0.013204,0.004640,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.005285,0.017539,-0.050515,0.013451,0.003757,0.015802,0.006859,0.002451,0.004963,0.005835,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.000336,0.126616,0.012569,-0.004890,0.014399,-0.001261,0.019487,0.012922,-0.004688,-0.006880,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.004575,-0.006274,-0.039186,0.002691,-0.005549,0.009472,0.023155,0.022602,0.012247,0.000971,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-0.003148,0.019078,-0.011368,-0.020828,0.008451,-0.019183,-0.025972,-0.020155,-0.003475,0.006493,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.001483,-0.004618,-0.014559,0.003196,-0.014115,-0.024141,-0.007471,0.015471,0.008635,0.004462
57,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.015996,-0.000196,0.000572,0.001816,0.008850,0.020964,0.013428,-0.020974,0.002450,0.005667
58,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.005419,0.001634,-0.013535,-0.011299,0.002458,0.004928,0.019474,0.000202,-0.007799,0.010204
59,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.008758,-0.000587,0.006059,0.011003,0.008918,0.022477,0.000656,0.016569,0.010739,0.006633


In [8]:
covA = returnsA.cov()
covB = returnsB.cov()
covC = returnsC.cov()
cov = returns.cov()
meanA = returnsA.mean()
meanB = returnsB.mean()
meanC = returnsC.mean()
meanT = returns.mean()
portA_mean = meanA.dot(np.array(portA.Holding))
portB_mean = meanB.dot(np.array(portB.Holding))
portC_mean = meanC.dot(np.array(portC.Holding))
port_mean = meanT.dot(np.array(port.Holding))

In [9]:
# Calculate portfolio standard deviation
portA_stdev = np.sqrt(np.array(portA.Holding).T.dot(covA).dot(np.array(portA.Holding)))
portB_stdev = np.sqrt(np.array(portB.Holding).T.dot(covB).dot(np.array(portB.Holding)))
portC_stdev = np.sqrt(np.array(portC.Holding).T.dot(covC).dot(np.array(portC.Holding)))
port_stdev = np.sqrt(np.array(port.Holding).T.dot(cov).dot(np.array(port.Holding)))

In [10]:
# Select our confidence interval (I'll choose 95% here)
conf_level1 = 0.05
cutoffA = norm.ppf(conf_level1, portA_mean, portA_stdev)
cutoffB = norm.ppf(conf_level1, portB_mean, portB_stdev)
cutoffC = norm.ppf(conf_level1, portC_mean, portC_stdev)
cutoffT = norm.ppf(conf_level1, port_mean, port_stdev)

In [11]:
varA = 1 - cutoffA
varB = 1 - cutoffB
varC = 1 - cutoffC
varT = 1 - cutoffT

In [12]:
varA

1.013918537073878

In [13]:
varB

1.0144714479417678

In [14]:
varC

1.0110980676399755

In [15]:
varT

1.0053690699731563